# **FYP Data Cleaning**

**Import Module**

All the import module will be written below

In [1]:
import pandas as pd
from tabulate import tabulate
from scipy.stats import skew, kurtosis
import numpy as np
import holidays
import seaborn as sns
import matplotlib.pyplot as plt


**Function for Statistical Analysis (Class Variable)**

In [2]:
def categorical_summary(df):
    summary = []

    for col in df.select_dtypes(include='object').columns:
        value_counts = df[col].value_counts(dropna=False)
        n_missing = df[col].isnull().sum()
        n_levels = df[col].nunique(dropna=True)

        mode1 = value_counts.index[0]
        mode1_pct = round(100 * value_counts.iloc[0] / len(df), 2)

        if len(value_counts) > 1:
            mode2 = value_counts.index[1]
            mode2_pct = round(100 * value_counts.iloc[1] / len(df), 2)
        else:
            mode2 = None
            mode2_pct = None

        summary.append({
            'Variable Name': col,
            'Levels': n_levels,
            'Missing': n_missing,
            'Mode': mode1,
            'Mode %': mode1_pct,
            'Mode2': mode2,
            'Mode2 %': mode2_pct
        })

    return pd.DataFrame(summary)

**Function for Statistical Analysis (Interval Variable)**

In [3]:
def interval_summary(df):
    summary = []

    numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns

    for col in numeric_cols:
        data = df[col].dropna()
        n_missing = df[col].isnull().sum()

        summary.append({
            'Variable': col,
            'Mean': round(data.mean(), 4),
            'Std Dev': round(data.std(), 4),
            'Missing': n_missing,
            'Min': data.min(),
            'Median': data.median(),
            'Max': data.max(),
            'Skewness': round(skew(data), 4),
            'Kurtosis': round(kurtosis(data), 4)
        })

    return pd.DataFrame(summary)

**Function for Statistical Analysis (DateTime Variable)**

In [4]:
def datetime_summary(df):
    summary = []

    for col in df.select_dtypes(include='datetime').columns:
        n_missing = df[col].isnull().sum()
        min_date = df[col].min()
        max_date = df[col].max()

        summary.append({
            'Variable Name': col,
            'Type': 'datetime',
            'Missing': n_missing,
            'Earliest': min_date,
            'Latest': max_date,
        })

    return pd.DataFrame(summary)

**olist_products_dataset.csv**

Read olist_products_dataset.csv

In [5]:
product_df = pd.read_csv("data/olist_products_dataset.csv")
product_df

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


Remove Row with Empty product_category_name

In [6]:
empty_rows = product_df[
    (product_df['product_category_name'].isna() | (product_df['product_category_name'].str.strip() == '')) &
    (product_df['product_name_lenght'].isna() | (product_df['product_name_lenght'] == 0)) &
    (product_df['product_description_lenght'].isna() | (product_df['product_description_lenght'] == 0))
]

print(f"Total rows with empty category, name length, and description length: {len(empty_rows)}")


Total rows with empty category, name length, and description length: 610


In [7]:
product_df = product_df.drop(empty_rows.index)
product_df

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,moveis_decoracao,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construcao_ferramentas_iluminacao,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,cama_mesa_banho,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,informatica_acessorios,60.0,156.0,2.0,700.0,31.0,13.0,20.0


Replace product category language

In [8]:
product_translation_df = pd.read_csv("data/product_category_name_translation.csv")
product_translation_df

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor
...,...,...
66,flores,flowers
67,artes_e_artesanato,arts_and_craftmanship
68,fraldas_higiene,diapers_and_hygiene
69,fashion_roupa_infanto_juvenil,fashion_childrens_clothes


In [9]:
translation_dict = dict(zip(
    product_translation_df['product_category_name'],
    product_translation_df['product_category_name_english']
))
product_df['product_category_name'] = product_df['product_category_name'].map(translation_dict).fillna(product_df['product_category_name'])
product_df

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,baby,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,housewares,37.0,402.0,4.0,625.0,20.0,17.0,13.0
...,...,...,...,...,...,...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,furniture_decor,45.0,67.0,2.0,12300.0,40.0,40.0,40.0
32947,bf4538d88321d0fd4412a93c974510e6,construction_tools_lights,41.0,971.0,1.0,1700.0,16.0,19.0,16.0
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,bed_bath_table,50.0,799.0,1.0,1400.0,27.0,7.0,27.0
32949,83808703fc0706a22e264b9d75f04a2e,computers_accessories,60.0,156.0,2.0,700.0,31.0,13.0,20.0


Drop Unused Column

In [10]:
product_df = product_df[['product_id', 'product_category_name']]
product_df

,product_id,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,art
2,96bd76ec8810374ed1b65e291975717f,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,baby
4,9dc1a7de274444849c219cff195d0b71,housewares
...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,furniture_decor
32947,bf4538d88321d0fd4412a93c974510e6,construction_tools_lights
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,bed_bath_table
32949,83808703fc0706a22e264b9d75f04a2e,computers_accessories


Statistical Analysis (Class Variable) After Cleaning

In [11]:
cat_summary = categorical_summary(product_df)
print(tabulate(cat_summary, headers='keys', tablefmt='pretty', showindex=False))

+-----------------------+--------+---------+----------------------------------+--------+----------------------------------+---------+
|     Variable Name     | Levels | Missing |               Mode               | Mode % |              Mode2               | Mode2 % |
+-----------------------+--------+---------+----------------------------------+--------+----------------------------------+---------+
|      product_id       | 32341  |    0    | 1e9e8ef04dbcff4541ed26657ea517e5 |  0.0   | 3aa071139cb16b67ca9e5dea641aaa2f |   0.0   |
| product_category_name |   73   |    0    |          bed_bath_table          |  9.37  |          sports_leisure          |  8.86   |
+-----------------------+--------+---------+----------------------------------+--------+----------------------------------+---------+


**olist_orders_dataset.csv**

Read olist_orders_dataset

In [12]:
order_df = pd.read_csv("data/olist_orders_dataset.csv")
order_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00


Replace Data Type

In [13]:
order_df['order_purchase_timestamp'] = pd.to_datetime(order_df['order_purchase_timestamp'])
order_df['order_approved_at'] = pd.to_datetime(order_df['order_approved_at'])
order_df['order_delivered_carrier_date'] = pd.to_datetime(order_df['order_delivered_carrier_date'])
order_df['order_delivered_customer_date'] = pd.to_datetime(order_df['order_delivered_customer_date'])
order_df['order_estimated_delivery_date'] = pd.to_datetime(order_df['order_estimated_delivery_date'])
order_df.dtypes

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

Remove Unlogic Row

In [14]:
order_df = order_df[
    ~(
        (order_df['order_status'] == 'delivered') &
        (order_df['order_approved_at'].isna() | (order_df['order_approved_at'].astype(str).str.strip() == ''))
    )
]
order_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15


In [15]:
order_df = order_df[
    ~(
        (order_df['order_status'] == 'delivered') &
        (order_df['order_delivered_carrier_date'].isna() | (order_df['order_delivered_carrier_date'].astype(str).str.strip() == ''))
    )
]
order_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15


In [16]:
order_df = order_df[
    ~(
        (order_df['order_status'] == 'delivered') &
        (order_df['order_delivered_customer_date'].isna() | (order_df['order_delivered_customer_date'].astype(str).str.strip() == '')) &
    ~(order_df['order_delivered_carrier_date'].isna() | (order_df['order_delivered_carrier_date'].astype(str).str.strip() == ''))
    )
]
order_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26
...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15


**olist_order_items_dataset.csv**

Read olist_order_items_dataset

In [17]:
order_items_df = pd.read_csv("data/olist_order_items_dataset.csv")
order_items_df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


Replace Data Type

In [18]:
order_items_df['shipping_limit_date'] = pd.to_datetime(order_items_df['shipping_limit_date'])
order_items_df.dtypes

order_id                       object
order_item_id                   int64
product_id                     object
seller_id                      object
shipping_limit_date    datetime64[ns]
price                         float64
freight_value                 float64
dtype: object

Remove Row with Product ID that not exists

In [19]:
order_items_df = order_items_df[
    order_items_df['product_id'].isin(product_df['product_id'])
]
order_items_df = order_items_df.reset_index(drop=True)
order_items_df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
111042,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
111043,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
111044,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
111045,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


Remove Row with Order ID that not exists

In [20]:
order_items_df = order_items_df[
    order_items_df['order_id'].isin(order_df['order_id'])
]
order_items_df = order_items_df.reset_index(drop=True)
order_items_df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
111019,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
111020,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
111021,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
111022,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [21]:
cat_summary = categorical_summary(order_items_df)
print(tabulate(cat_summary, headers='keys', tablefmt='pretty', showindex=False))

+---------------+--------+---------+----------------------------------+--------+----------------------------------+---------+
| Variable Name | Levels | Missing |               Mode               | Mode % |              Mode2               | Mode2 % |
+---------------+--------+---------+----------------------------------+--------+----------------------------------+---------+
|   order_id    | 97255  |    0    | 8272b63d03f5f79c56e9e4120aec44ef |  0.02  | ab14fdcfbe524636d65ee38360e22ce8 |  0.02   |
|  product_id   | 32335  |    0    | aca2eb7d00ea1a7b8ebd4e68314663af |  0.47  | 99a4788cb24856965c36a24e339b6058 |  0.44   |
|   seller_id   |  3035  |    0    | 6560211a19b47992c3666cc44a7e94c0 |  1.83  | 4a3ca9315b744ce9f8e9374361493884 |  1.79   |
+---------------+--------+---------+----------------------------------+--------+----------------------------------+---------+


In [22]:
int_summary = interval_summary(order_items_df)
print(tabulate(int_summary, headers='keys', tablefmt='pretty', showindex=False))

+---------------+----------+----------+---------+------+--------+--------+----------+----------+
|   Variable    |   Mean   | Std Dev  | Missing | Min  | Median |  Max   | Skewness | Kurtosis |
+---------------+----------+----------+---------+------+--------+--------+----------+----------+
| order_item_id |  1.1982  |  0.7069  |    0    | 1.0  |  1.0   |  21.0  |  7.5948  | 104.0863 |
|     price     | 120.7762 | 183.4337 |    0    | 0.85 | 74.99  | 6735.0 |  7.8847  | 120.4042 |
| freight_value | 20.0256  | 15.8419  |    0    | 0.0  | 16.305 | 409.68 |  5.6418  | 59.8572  |
+---------------+----------+----------+---------+------+--------+--------+----------+----------+


In [23]:
date_summary = datetime_summary(order_items_df)
print(tabulate(date_summary, headers='keys', tablefmt='pretty', showindex=False))

+---------------------+----------+---------+---------------------+---------------------+
|    Variable Name    |   Type   | Missing |      Earliest       |       Latest        |
+---------------------+----------+---------+---------------------+---------------------+
| shipping_limit_date | datetime |    0    | 2016-09-19 00:15:34 | 2020-04-09 22:35:08 |
+---------------------+----------+---------+---------------------+---------------------+


Remove Row from order dataset with Order ID that not exists in order item dataset

In [24]:
order_df = order_df[
    order_df['order_id'].isin(order_items_df['order_id'])
]
order_df = order_df.reset_index(drop=True)
order_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26
...,...,...,...,...,...,...,...,...
97250,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28
97251,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02
97252,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27
97253,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15


In [25]:
date_summary = datetime_summary(order_df)
print(tabulate(date_summary, headers='keys', tablefmt='pretty', showindex=False))

+-------------------------------+----------+---------+---------------------+---------------------+
|         Variable Name         |   Type   | Missing |      Earliest       |       Latest        |
+-------------------------------+----------+---------+---------------------+---------------------+
|   order_purchase_timestamp    | datetime |    0    | 2016-09-04 21:15:19 | 2018-09-03 09:06:57 |
|       order_approved_at       | datetime |    0    | 2016-09-15 12:16:38 | 2018-09-03 17:40:06 |
| order_delivered_carrier_date  | datetime |   976   | 2016-10-08 10:34:01 | 2018-09-11 19:48:28 |
| order_delivered_customer_date | datetime |  2125   | 2016-10-11 13:46:32 | 2018-10-17 13:22:46 |
| order_estimated_delivery_date | datetime |    0    | 2016-10-04 00:00:00 | 2018-10-25 00:00:00 |
+-------------------------------+----------+---------+---------------------+---------------------+


Create quality column for product

In [26]:
quantity_counts = order_items_df['product_id'].value_counts()
product_df['quantity'] = product_df['product_id'].map(quantity_counts).fillna(0).astype(int)
product_df

C:\Users\ASUS\AppData\Local\Temp\ipykernel_46000\3232233087.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  product_df['quantity'] = product_df['product_id'].map(quantity_counts).fillna(0).astype(int)


,product_id,product_category_name,quantity
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,1
1,3aa071139cb16b67ca9e5dea641aaa2f,art,1
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,1
3,cef67bcfe19066a932b7673e239eb23d,baby,1
4,9dc1a7de274444849c219cff195d0b71,housewares,1
...,...,...,...
32946,a0b7d5a992ccda646f2d34e418fff5a0,furniture_decor,33
32947,bf4538d88321d0fd4412a93c974510e6,construction_tools_lights,2
32948,9a7c6041fa9592d9d9ef6cfe62a71f8c,bed_bath_table,5
32949,83808703fc0706a22e264b9d75f04a2e,computers_accessories,7


Save Cleaned Data

In [27]:
product_df.to_csv('Product Data.csv', index=False)
order_df.to_csv('Order Data.csv', index=False)
order_items_df.to_csv('Order Items Data.csv', index=False)

**Sales Forecasting Feature Engineering**

**Preparing Sales Forecasting Dataset**

Filter invalid status

In [28]:
valid_statuses = [
    'delivered',
    'shipped',
    'invoiced',
    'processing',
    'approved'
]
print(order_df['order_status'].unique())

sales_forecast_data = order_df[order_df['order_status'].isin(valid_statuses)].copy()
sales_forecast_data


['delivered' 'shipped' 'invoiced' 'processing' 'canceled' 'unavailable'
 'approved']


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26
...,...,...,...,...,...,...,...,...
97250,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28
97251,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02
97252,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27
97253,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15


Data Aggregation

In [29]:
sales_forecast_data_filtered = order_items_df[
    order_items_df['order_id'].isin(sales_forecast_data['order_id'])
].reset_index(drop=True)

sales_forecast_data_filtered

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
110484,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
110485,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
110486,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
110487,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [30]:
merged_df = sales_forecast_data_filtered.merge(
    sales_forecast_data[['order_id', 'order_purchase_timestamp']],
    on='order_id',
    how='left'
)

merged_df['order_date'] = pd.to_datetime(merged_df['order_purchase_timestamp']).dt.date

grouped_sales_forecast_data = merged_df.groupby(['product_id', 'order_date']).agg(
    quantity=('order_id', 'count'),  # Number of orders per product per day
).reset_index()

grouped_sales_forecast_data = grouped_sales_forecast_data.sort_values(['order_date']).reset_index(drop=True)

grouped_sales_forecast_data

,product_id,order_date,quantity
0,c1488892604e4ba5cff5b4eb4d595400,2016-09-04,1
1,f293394c72c9b5fafd7023301fc21fc2,2016-09-04,1
2,5a6b04657a4c5ee34285d1e4619a96b4,2016-09-15,3
3,107177bf61755f05c604fe57e02467d6,2016-10-03,1
4,3ae08df6bcbfe23586dd431c40bddbb7,2016-10-03,1
...,...,...,...
92830,9a8706b8c060b16e5f0d2925f20bc35b,2018-08-29,1
92831,9865c67a74684715521d1e70226cce0b,2018-08-29,1
92832,2b4472df15512a2825ae86fd9ae79335,2018-08-29,2
92833,d04857e7b4b708ee8b8b9921163edba3,2018-08-29,1


Define Holiday

In [31]:
br_holidays = holidays.Brazil()
grouped_sales_forecast_data['is_holiday'] = grouped_sales_forecast_data['order_date'].apply(lambda x: 1 if x in br_holidays else 0)
grouped_sales_forecast_data

,product_id,order_date,quantity,is_holiday
0,c1488892604e4ba5cff5b4eb4d595400,2016-09-04,1,0
1,f293394c72c9b5fafd7023301fc21fc2,2016-09-04,1,0
2,5a6b04657a4c5ee34285d1e4619a96b4,2016-09-15,3,0
3,107177bf61755f05c604fe57e02467d6,2016-10-03,1,0
4,3ae08df6bcbfe23586dd431c40bddbb7,2016-10-03,1,0
...,...,...,...,...
92830,9a8706b8c060b16e5f0d2925f20bc35b,2018-08-29,1,0
92831,9865c67a74684715521d1e70226cce0b,2018-08-29,1,0
92832,2b4472df15512a2825ae86fd9ae79335,2018-08-29,2,0
92833,d04857e7b4b708ee8b8b9921163edba3,2018-08-29,1,0


Extracting Time-Based Features from Date

In [32]:
grouped_sales_forecast_data['order_date'] = pd.to_datetime(grouped_sales_forecast_data['order_date'])

grouped_sales_forecast_data['day'] = grouped_sales_forecast_data['order_date'].dt.day
grouped_sales_forecast_data['day_of_week'] = grouped_sales_forecast_data['order_date'].dt.dayofweek  # Monday=0, Sunday=6
grouped_sales_forecast_data['week'] = grouped_sales_forecast_data['order_date'].dt.isocalendar().week.astype(int)
grouped_sales_forecast_data['month'] = grouped_sales_forecast_data['order_date'].dt.month
grouped_sales_forecast_data['quarter'] = grouped_sales_forecast_data['order_date'].dt.quarter
grouped_sales_forecast_data['year'] = grouped_sales_forecast_data['order_date'].dt.year
grouped_sales_forecast_data['is_weekend'] = grouped_sales_forecast_data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)

grouped_sales_forecast_data

,product_id,order_date,quantity,is_holiday,day,day_of_week,week,month,quarter,year,is_weekend
0,c1488892604e4ba5cff5b4eb4d595400,2016-09-04,1,0,4,6,35,9,3,2016,1
1,f293394c72c9b5fafd7023301fc21fc2,2016-09-04,1,0,4,6,35,9,3,2016,1
2,5a6b04657a4c5ee34285d1e4619a96b4,2016-09-15,3,0,15,3,37,9,3,2016,0
3,107177bf61755f05c604fe57e02467d6,2016-10-03,1,0,3,0,40,10,4,2016,0
4,3ae08df6bcbfe23586dd431c40bddbb7,2016-10-03,1,0,3,0,40,10,4,2016,0
...,...,...,...,...,...,...,...,...,...,...,...
92830,9a8706b8c060b16e5f0d2925f20bc35b,2018-08-29,1,0,29,2,35,8,3,2018,0
92831,9865c67a74684715521d1e70226cce0b,2018-08-29,1,0,29,2,35,8,3,2018,0
92832,2b4472df15512a2825ae86fd9ae79335,2018-08-29,2,0,29,2,35,8,3,2018,0
92833,d04857e7b4b708ee8b8b9921163edba3,2018-08-29,1,0,29,2,35,8,3,2018,0


Creating Lag and Rolling Mean Features for Temporal Dependencies

In [33]:
grouped_sales_forecast_data = grouped_sales_forecast_data.sort_values(['product_id', 'order_date'])

grouped_sales_forecast_data['qty_lag_1'] = grouped_sales_forecast_data.groupby('product_id')['quantity'].shift(1).fillna(0)
grouped_sales_forecast_data['qty_lag_2'] = grouped_sales_forecast_data.groupby('product_id')['quantity'].shift(2).fillna(0)
grouped_sales_forecast_data['qty_lag_3'] = grouped_sales_forecast_data.groupby('product_id')['quantity'].shift(3).fillna(0)
grouped_sales_forecast_data['qty_lag_7'] = grouped_sales_forecast_data.groupby('product_id')['quantity'].shift(7).fillna(0)

grouped_sales_forecast_data['qty_rolling_mean_3'] = grouped_sales_forecast_data.groupby('product_id')['quantity'].transform(lambda x: x.shift(1).rolling(window=3, min_periods=1).mean()).fillna(0)
grouped_sales_forecast_data['qty_rolling_mean_7'] = grouped_sales_forecast_data.groupby('product_id')['quantity'].transform(lambda x: x.shift(1).rolling(window=7, min_periods=1).mean()).fillna(0)

grouped_sales_forecast_data['qty_pct_change'] = grouped_sales_forecast_data.groupby('product_id')['quantity'].pct_change().fillna(0)
grouped_sales_forecast_data['cumulative_qty'] = grouped_sales_forecast_data.groupby('product_id')['quantity'].cumsum()

grouped_sales_forecast_data

,product_id,order_date,quantity,is_holiday,day,day_of_week,week,month,quarter,year,is_weekend,qty_lag_1,qty_lag_2,qty_lag_3,qty_lag_7,qty_rolling_mean_3,qty_rolling_mean_7,qty_pct_change,cumulative_qty
73096,00066f42aeeb9f3007548bb9d3f33c38,2018-05-20,1,0,20,6,20,5,2,2018,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
39589,00088930e925c41fd95ebfe695fd2655,2017-12-12,1,0,12,1,50,12,4,2017,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
41257,0009406fd7479715e4bef61dd91f2462,2017-12-21,1,0,21,3,51,12,4,2017,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
86632,000b8f95fcb9e0096488278317764d19,2018-08-01,1,0,1,2,31,8,3,2018,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
89445,000b8f95fcb9e0096488278317764d19,2018-08-10,1,0,10,4,32,8,3,2018,0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65752,fffdb2d0ec8d6a61f0a0a0db3f25b441,2018-04-19,1,0,19,3,16,4,2,2018,0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,2
68464,fffdb2d0ec8d6a61f0a0a0db3f25b441,2018-05-01,1,1,1,1,18,5,2,2018,0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,3
88391,fffdb2d0ec8d6a61f0a0a0db3f25b441,2018-08-07,1,0,7,1,32,8,3,2018,0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,4
89346,fffdb2d0ec8d6a61f0a0a0db3f25b441,2018-08-10,1,0,10,4,32,8,3,2018,0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,5


In [34]:
grouped_sales_forecast_data.to_csv("Sales Forecasting Data.csv", index=False)